[사용 데이터](https://public.roboflow.com/object-detection/pklot/1/images/578a957dbf5315af05ee2bd278dcf6d6)

In [15]:
import tensorflow.keras

In [21]:
import os, glob
import scipy
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [22]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)

2022-03-15 14:30:48.450258: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-15 14:30:48.453237: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-15 14:30:48.497478: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-15 14:30:48.498066: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-03-15 14:30:48.498094: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-15 14:30:48.538308: I tensorflow/stream_executor/platform/default/dso_loade

In [39]:
#clone yolo v5

%mkdir /home/lab27/yolov5/pklot
%cd /home/lab27/yolov5/pklot

# !git clone https://github.com/ultralytics/yolov5.git

/home/lab27/yolov5/pklot


In [40]:
!curl -L "https://public.roboflow.com/ds/fdKahW2Lel?key=ifb9z08mRA" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0   1563      0 --:--:-- --:--:-- --:--:--  1563
100  866M  100  866M    0     0  80.7M      0  0:00:10  0:00:10 --:--:--  106M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/2012-09-11_15_53_00_jpg.rf.8282544a640a23df05bd245a9210e663.jpg  
 extracting: test/images/2012-09-11_16_48_36_jpg.rf.4ecc8c87c61680ccc73edc218a2c8d7d.jpg  
 extracting: test/images/2012-09-12_06_36_36_jpg.rf.08869047c7e9f62f5ce9334546b52958.jpg  
 extracting: test/images/2012-09-12_08_15_53_jpg.rf.025635f821fe24c12ab497efbfd3d35c.jpg  
 extracting: test/images/2012-09-12_09_18_46_jpg.rf.d95bc1a314fb384813111f94c25b4ef3.jpg  
 extracting: test/images/2012-09-12_10_05_57_jp

In [37]:
%cd /home/lab27

# Git clone
!git clone https://github.com/ultralytics/yolov5.git

/home/lab27
Cloning into 'yolov5'...
remote: Enumerating objects: 11301, done.
remote: Total 11301 (delta 0), reused 0 (delta 0), pack-reused 11301
Receiving objects: 100% (11301/11301), 11.16 MiB | 11.09 MiB/s, done.
Resolving deltas: 100% (7818/7818), done.


In [38]:
!pip install -r /home/lab27/yolov5/requirements.txt

In [35]:
from glob import glob
from sklearn.model_selection import train_test_split
import yaml

In [34]:
!pip install sklearn

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 72.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 KB 41.8 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=dde6240f5993d9cfa33bea81b2ee54c144fbfd060c0068c9b80e5c88cc8145d6
  Stored in directory: /home/lab27/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [41]:
train_img_list=glob('/home/lab27/yolov5/pklot/train/images/*.jpg')
test_img_list=glob('/home/lab27/yolov5/pklot/test/images/*.jpg')
valid_img_list=glob('/home/lab27/yolov5/pklot/valid/images/*.jpg')
print(len(train_img_list),len(test_img_list),len(valid_img_list))

8691 1242 2483


In [47]:
import yaml

with open('/home/lab27/yolov5/pklot/data.yaml','r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/home/lab27/yolov5/pklot/'
data['test'] = '/home/lab27/yolov5/pklot/'
data['val'] = '/home/lab27/yolov5/pklot/'

with open('/home/lab27/yolov5/pklot/data.yaml', 'w') as f:
  yaml.dump(data, f)


print(data)


{'train': '../train/images', 'val': '../valid/images', 'nc': 2, 'names': ['space-empty', 'space-occupied']}
{'train': '/home/lab27/yolov5/pklot/', 'val': '/home/lab27/yolov5/pklot/', 'nc': 2, 'names': ['space-empty', 'space-occupied'], 'test': '/home/lab27/yolov5/pklot/'}


In [50]:
%cd /home/lab27/yolov5/

!python train.py --img 640 --batch 32 --epochs 20 --data ./pklot/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name pklot_results

/home/lab27/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=./pklot/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=pklot_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-35-g932dc78 torch 1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_

In [51]:
%load_ext tensorboard
%tensorboard --logdir /home/lab27/yolov5/runs/

In [52]:
%cd /home/lab27/yolov5/

from glob import glob

img_list = glob('/home/lab27/yolov5/pklot/test/images/*.jpg')

print(len(img_list))

/home/lab27/yolov5
1242


In [53]:
from IPython.display import Image
import os

In [54]:
val_img_path = img_list[7]

weights_path = '/home/lab27/yolov5/runs/train/pklot_results/weights/best.pt'

!python detect.py --weights "{weights_path}" --img 416 --conf 0.5 --source "{val_img_path}"

detect_img_path = '/content/yolov5/runs/detect/exp'

Image(os.path.join(detect_img_path, os.path.basename(val_img_path)))

detect: weights=['/home/lab27/yolov5/runs/train/pklot_results/weights/best.pt'], source=/home/lab27/yolov5/pklot/test/images/2012-10-16_10_53_34_jpg.rf.853c37badf68065821bebf54b256167a.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-35-g932dc78 torch 1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /home/lab27/yolov5/pklot/test/images/2012-10-16_10_53_34_jpg.rf.853c37badf68065821bebf54b256167a.jpg: 416x416 12 space-emptys, 88 space-occupieds, Done. (0.009s)
Speed: 0.4ms pre-process, 8.8ms inference, 1.5ms NMS per image at shape (1, 3, 416, 416)
Resul

FileNotFoundError: No such file or directory: '/content/yolov5/runs/detect/exp/2012-10-16_10_53_34_jpg.rf.853c37badf68065821bebf54b256167a.jpg'

FileNotFoundError: No such file or directory: '/content/yolov5/runs/detect/exp/2012-10-16_10_53_34_jpg.rf.853c37badf68065821bebf54b256167a.jpg'

<IPython.core.display.Image object>